<a href="https://colab.research.google.com/github/BluRaf/tree-classify/blob/master/basic-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install tflite-model-maker

     |████████████████████████████████| 194kB 7.5MB/s 
     |████████████████████████████████| 1.0MB 12.5MB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 5.5MB 13.5MB/s 
     |████████████████████████████████| 1.2MB 53.7MB/s 
     |████████████████████████████████| 276kB 52.2MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 849kB 40.8MB/s 
     |████████████████████████████████| 194kB 49.2MB/s 
     |████████████████████████████████| 174kB 56.0MB/s 
     |████████████████████████████████| 37.6MB 97kB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 358kB 43.9MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=c08a5acd7cc60f82915178758c1e1438792955723ca215a87a42892681e9cc11
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Create

In [12]:
import os
from collections import namedtuple
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker import model_spec

import matplotlib.pyplot as plt

In [3]:
# Count images in dataset
import pathlib
data = pathlib.Path('/content/drive/MyDrive/studia/sem5/pk/dataset') 
count = len(list(data.glob('*/*.jpg')))
count

1647

In [4]:
# Use built-in data loader to load dataset
data = ImageClassifierDataLoader.from_folder("/content/drive/MyDrive/studia/sem5/pk/dataset")
train_data, rest_data = data.split(0.7)
validation_data, test_data = rest_data.split(0.5)

INFO:tensorflow:Load image with size: 1647, num_label: 5, labels: akacja, brzoza, dab, lipa, wierzba.


In [5]:
# Create models based on available pre-trained models

# EfficientNet-Lite0
model_effnetl0 = image_classifier.create(
    train_data, 
    validation_data=validation_data
    )
# MobileNetV2
model_mobnetv2 = image_classifier.create(
    train_data, validation_data=validation_data, 
    model_spec=model_spec.mobilenet_v2_spec
    )
# ResNet-50
model_resnet50 = image_classifier.create(
    train_data, validation_data=validation_data, 
    model_spec=model_spec.resnet_50_spec
    )

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
36/36 [==============================] - 466s 13s/step - loss: 1.2789 - accuracy: 0.4980 - val_loss: 0.5916 - val_accuracy: 0.9420
Epoch 2/5
36/36 [==============================] - 64s 2s/step - loss: 0.5941 - accuracy: 0.9446 - val_loss: 0.5212 - val_accuracy: 0.9866
Epoch 3/5
36/36 [=============================

INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_1 (HubK (None, 1280)              2257984   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________
None
Epoch 1/5
36/36 [==============================] - 55s 1s/step - loss: 1.3335 - accuracy: 0.5149 - val_loss: 0.5863 - val_accuracy: 0.9509
Epoch 2/5
36/36 [==============================] - 51s 1s/step - loss: 0.6016 - accuracy: 0.9390 - val_loss: 0.5135 - val_accuracy: 0.9866
Epoch 3/5
36/36 [==============================] - 51s 1s/step - loss: 0.5458 - accurac

INFO:tensorflow:Retraining the models...


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_2 (HubK (None, 2048)              23564800  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 10245     
Total params: 23,575,045
Trainable params: 10,245
Non-trainable params: 23,564,800
_________________________________________________________________
None
Epoch 1/5
36/36 [==============================] - 126s 3s/step - loss: 1.2475 - accuracy: 0.5600 - val_loss: 0.6489 - val_accuracy: 0.9509
Epoch 2/5
36/36 [==============================] - 122s 3s/step - loss: 0.6542 - accuracy: 0.9177 - val_loss: 0.5571 - val_accuracy: 0.9598
Epoch 3/5
36/36 [==============================] - 122s 3s/step - loss: 0.5653 - a

In [11]:
# Evaluate trained models
Model = namedtuple('Model', 'name model')
models = (
    Model('EfficientNet-Lite0', model_effnetl0),
    Model('MobileNetV2', model_mobnetv2),
    Model('ResNet-50', model_resnet50)
)

for model_i in models:
    print(">>> Evaluating {}".format(model_i.name))
    loss, accuracy = model_i.model.evaluate(test_data)
    print("{}: Accuracy {}%, loss {}%".format(
        model_i.name, accuracy*100, loss*100))
    print("")

>>> Evaluating EfficientNet-Lite0
8/8 [==============================] - 14s 1s/step - loss: 0.4884 - accuracy: 0.9798
EfficientNet-Lite0: Accuracy 97.98387289047241%, loss 48.83814752101898%

>>> Evaluating MobileNetV2
8/8 [==============================] - 12s 1s/step - loss: 0.4993 - accuracy: 0.9798
MobileNetV2: Accuracy 97.98387289047241%, loss 49.934667348861694%

>>> Evaluating ResNet-50
8/8 [==============================] - 25s 3s/step - loss: 0.5449 - accuracy: 0.9355
ResNet-50: Accuracy 93.54838728904724%, loss 54.487234354019165%



In [17]:
# Export trained models

for model_i in models:
    print(">>> Exporting {}".format(model_i.name))
    dirpath = './{}'.format(model_i.name)
    os.makedirs(dirpath, exist_ok = True)
    model_i.model.export(export_dir=dirpath, with_metadata=False)

>>> Exporting EfficientNet-Lite0
INFO:tensorflow:Assets written to: /tmp/tmp4mfnxtpn/assets


INFO:tensorflow:Assets written to: /tmp/tmp4mfnxtpn/assets


INFO:tensorflow:Saving labels in ./EfficientNet-Lite0/labels.txt.


INFO:tensorflow:Saving labels in ./EfficientNet-Lite0/labels.txt.


>>> Exporting MobileNetV2
INFO:tensorflow:Assets written to: /tmp/tmprtn1adc8/assets


...

INFO:tensorflow:Saving labels in ./MobileNetV2/labels.txt.


>>> Exporting ResNet-50
INFO:tensorflow:Assets written to: /tmp/tmphptexj7d/assets


INFO:tensorflow:Assets written to: /tmp/tmphptexj7d/assets


INFO:tensorflow:Saving labels in ./ResNet-50/labels.txt.


INFO:tensorflow:Saving labels in ./ResNet-50/labels.txt.


In [19]:
# Evaluate exported models

for model_i in models:
    print(">>> Evaluating {}".format(model_i.name))
    model_i.model.evaluate_tflite('{}/model.tflite'.format(dirpath), test_data);

>>> Evaluating EfficientNet-Lite0
INFO:tensorflow:Processing example: #0
[[[[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  ...

  [[0.5316531  0.5551825  0.60224134]
   [0.53525406 0.5587835  0.6058423 ]
   [0.53469354 0.55822295 0.6052818 ]
   ...
   [0.563909   0.5408599  0.48035604]
   [0.6677885  0.65810454 0.62977356]
   [0.696759

INFO:tensorflow:Processing example: #0
[[[[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  ...

  [[0.5316531  0.5551825  0.60224134]
   [0.53525406 0.5587835  0.6058423 ]
   [0.53469354 0.55822295 0.6052818 ]
   ...
   [0.563909   0.5408599  0.48035604]
   [0.6677885  0.65810454 0.62977356]
   [0.69675916 0.69355756 0.69187707]]

  [[0.

>>> Evaluating MobileNetV2
>>> Evaluating ResNet-50
